In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from L_score import L_score

In [43]:
class HackatonDataset(Dataset):
    def __init__(self, X, y_df):
        """
        Konstruktor przyjmuje macierz cech (X) oraz DataFrame y_df,
        w którym kolumna 'label' to etykieta layoutu (0..9),
        a 'clicked' to informacja aposteriori (0 lub 1).
        """
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y_df["label"].values, dtype=torch.long)
        self.clicked = torch.tensor(y_df["clicked"].values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.clicked[idx]

In [44]:
class ClickModel(nn.Module):
    def __init__(self, input_dim):
        super(ClickModel, self).__init__()
        # Zakładamy, że label traktujemy jako pojedynczą cechę liczbową, więc dodajemy 1 do input_dim
        self.fc = nn.Linear(input_dim + 1, 1)

    def forward(self, x, layout_idx):
        """
        x: tensor o kształcie [batch_size, input_dim]
        layout_idx: tensor [batch_size] (typ long), czyli bezpośrednio numer layoutu
        """
        # Konwertujemy layout na typ float i rozszerzamy wymiar, by miał kształt [batch_size, 1]
        layout_feature = layout_idx.unsqueeze(1).float()
        # Łączymy wektor cech x z cechą layoutu
        combined = torch.cat([x, layout_feature], dim=1)
        # Przewidujemy prawdopodobieństwo kliknięcia
        p = torch.sigmoid(self.fc(combined))
        return p  # zwracamy tensor o kształcie [batch_size, 1]

In [33]:
def custom_loss_fn( ##### nowa dynamic_lambdas !!!!!
    layout_logits,   # [batch_size, 10]
    true_layout,     # [batch_size]
    clicked,         # [batch_size], 0/1
    X_batch,         # [batch_size, input_dim]
    click_model,
    lambda_val# model przewidujący p=clicked
):
    """
    Zwraca średni loss:
      - cross entropy dla layoutu
      + "ręcznie" liczona kara (lub bonus=0)
    """
    # 1) Obliczamy CE (per sample, a nie średnią)
    ce = F.cross_entropy(layout_logits, true_layout, reduction='none')  # [batch_size]

    # 2) Predykcja layoutu (argmax).
    layout_pred = torch.argmax(layout_logits, dim=1)  # [batch_size]

    # 3) Szacujemy prawdopodobieństwo kliknięcia p
    #    w zależności od X i *wybranego* layoutu
    p = click_model(X_batch, layout_pred).squeeze(1)  # [batch_size]
    p = torch.clamp(p, min=1e-5, max=1-1e-5)  # minimalna stabilizacja

    # 4) Obliczamy karę według zadanych reguł
    correct = (layout_pred == true_layout)  # tensor bool [batch_size]
    
    # Inicjujemy zerowy wektor kary
    penalty = torch.zeros_like(ce)

    # A) layout_pred != layout_true => kara = 1/p
    mask_incorrect = (~correct)
    penalty[mask_incorrect] = 1.0 / p[mask_incorrect]

    # B) layout_pred == layout_true i clicked=0 => kara=1
    #    layout_pred == layout_true i clicked=1 => kara=0
    mask_correct_zero = (correct & (clicked==0))
    penalty[mask_correct_zero] = 1.0
    
    # mask_correct_one = (correct & (clicked==1)) => kara pozostaje = 0

    total_loss_per_sample = ce + penalty * lambda_val

    return total_loss_per_sample.mean()

In [45]:
# --- 1. Wczytanie i przygotowanie danych ---

X_train_scaled = pd.read_csv("data/feature_engineering/processed_x_train.csv")
X_val_scaled = pd.read_csv("data/feature_engineering/processed_x_valid.csv")
y_train = pd.read_csv("data/y_train.csv")
y_val = pd.read_csv("data/y_valid.csv")

# Tworzymy zestawy danych i DataLoadery
train_dataset = HackatonDataset(X_train_scaled, y_train)
val_dataset   = HackatonDataset(X_val_scaled, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

# --- 3. Definicja modelu (logistyczna regresja) ---

import torch
import torch.nn as nn
import torch.nn.functional as F

class LayoutModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 300),
            nn.Sigmoid(),
            nn.Linear(300, num_classes),
            nn.Softmax(dim=1)  # Używamy Softmax na końcu, aby uzyskać prawdopodobieństwa
        )
    
    def forward(self, x):
        return self.net(x)

input_dim = X_train_scaled.shape[1]
num_classes = 10  # Zakładamy 10 unikalnych wartości layout_type
model = LayoutModel(input_dim, num_classes)

# Wybieramy optymalizator
optimizer = optim.Adam(model.parameters(), lr=0.032)

# --- 4. Definicja customowego lossu ---

import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader

# ...
# Załóżmy, że mamy wczytane:
#   X_train_scaled, X_val_scaled   (DataFrame lub gotowe macierze)
#   y_train, y_val                 (DataFrame z kolumnami: 'label' i 'clicked')

train_dataset = HackatonDataset(X_train_scaled, y_train)
val_dataset   = HackatonDataset(X_val_scaled, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

# Definiujemy modele
input_dim = X_train_scaled.shape[1]
layout_model = LayoutModel(input_dim, 10)
click_model  = ClickModel(input_dim)

# Optymalizator z parametrami obu modeli
optimizer = optim.Adam(
    list(layout_model.parameters()) + list(click_model.parameters()),
    lr=0.1
)

num_epochs = 10
for epoch in range(num_epochs):
    layout_model.train()
    click_model.train()
    
    all_losses = []
    for X_batch, y_batch, clicked_batch in train_loader:
        optimizer.zero_grad()
        # forward layout
        layout_logits = layout_model(X_batch)
        # nasz custom loss
        loss = custom_loss_fn(
            layout_logits, 
            y_batch, 
            clicked_batch,
            X_batch,
            click_model,
            lambda_val=10
        )
        loss.backward()
        optimizer.step()
        all_losses.append(loss.item())
   

# --- Walidacja ---
layout_model.eval()
click_model.eval()
val_losses = []
all_preds = []
with torch.no_grad():
    for X_batch, y_batch, clicked_batch in val_loader:
        layout_logits = layout_model(X_batch)
        loss = custom_loss_fn(layout_logits, y_batch, clicked_batch, X_batch, click_model, lambda_val=10)
        val_losses.append(loss.item())
        
        pred_layout = torch.argmax(layout_logits, dim=1)
        all_preds.extend(pred_layout.cpu().numpy())
avg_val_loss = np.mean(val_losses)

# --- 5. Pętla treningowa wykorzystująca custom loss ---

lambda_val = 10  # Ustalona wartość parametru lambda – można eksperymentować
num_epochs = 20

######### mozliwe ze tutaj do zmiany

for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    for batch in train_loader:
        X_batch, y_batch, clicked_batch = batch
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = custom_loss_fn(logits, y_batch, clicked_batch, X_batch, click_model, lambda_val)

        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    print(f"Epoch {epoch+1:2d}/{num_epochs}, Training Loss: {np.mean(epoch_losses):.4f}")
    
    ######### mozliwe ze tutaj do zmiany - koniec

# --- 6. Walidacja modelu ---

model.eval()
val_losses = []
all_preds = []
all_targets = []
all_clicked = []

with torch.no_grad():
    for batch in val_loader:
        X_batch, y_batch, clicked_batch = batch
        logits = model(X_batch)
        loss = custom_loss_fn(logits, y_batch, clicked_batch, X_batch, click_model, lambda_val)
        val_losses.append(loss.item())
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())
        all_clicked.extend(clicked_batch.cpu().numpy())
        
avg_val_loss = np.mean(val_losses)
print(f"\nValidation Loss: {avg_val_loss:.4f}")

# --- 7. Ocena modelu za pomocą metryki L_score ---


# Przygotowanie zbioru walidacyjnego do obliczania L_score
val_df = y_val.copy()
# Dodajemy kolumnę z predykcjami uzyskanymi z modelu
val_df['y_pred'] = all_preds

l_score_val = L_score(val_df[['label','clicked']], val_df['y_pred'])
print(f"L_score na zbiorze walidacyjnym: {l_score_val:.4f}")

Epoch  1/20, Training Loss: 11.9942
Epoch  2/20, Training Loss: 11.9941
Epoch  3/20, Training Loss: 11.9941
Epoch  4/20, Training Loss: 11.9941
Epoch  5/20, Training Loss: 11.9941
Epoch  6/20, Training Loss: 11.9941
Epoch  7/20, Training Loss: 11.9941
Epoch  8/20, Training Loss: 11.9942
Epoch  9/20, Training Loss: 11.9941
Epoch 10/20, Training Loss: 11.9941
Epoch 11/20, Training Loss: 11.9941
Epoch 12/20, Training Loss: 11.9942
Epoch 13/20, Training Loss: 11.9941
Epoch 14/20, Training Loss: 11.9942
Epoch 15/20, Training Loss: 11.9942
Epoch 16/20, Training Loss: 11.9941
Epoch 17/20, Training Loss: 11.9942
Epoch 18/20, Training Loss: 11.9941
Epoch 19/20, Training Loss: 11.9941
Epoch 20/20, Training Loss: 11.9941

Validation Loss: 11.2924
L_score na zbiorze walidacyjnym: 0.1016
